In [ ]:
pip install --upgrade transformers

In [ ]:
pip install --upgrade huggingface-hub


In [3]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# 토크나이저랑 모델 불러오기
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]

# 입력 문장 설정 "pt"는 PyTorch
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# 레이블 설정
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters()) # 옵티마이저 등록
loss = model(**batch).loss # 손실계산 (loss 계산)
loss.backward()  # 역전파 수행 (모델의 가중치 업데이트하는 역할)
optimizer.step() # 옵티마이저가 모델의 파라미터를 업데이트

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
pip install datasets

In [5]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [6]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [7]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [8]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

In [9]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

In [11]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

In [12]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [13]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [14]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

In [16]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

In [17]:
# datasets으로부터 load_dataset 함수 호출
from datasets import load_dataset
# transformers로부터 AutoTokenizer, DataCollatorWithPadding 클래스 호출
from transformers import AutoTokenizer, DataCollatorWithPadding

# glue 데이터 셋 중에 mrpc 데이터셋 로드하여 raw_datasets에 저장 (자연어처리를 위해서)
raw_datasets = load_dataset("glue", "mrpc")
# BERT모델의 체크포인트를 bert-base-uncased로 설정
checkpoint = "bert-base-uncased"
# 지정한 체크포인트를 사용하여 BERT모델을 위한 토크나이저 초기화 후 tokenizer변수에 저장
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# 데이터셋의 각 예제를 입력으로 받아서 두개의 문장을 BERT 토큰으로 변환
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True) # truncation은 BERT모델 최대 길이에 맞게 길이 제한을 두는 것 / padding은 가져와서 그 자리만큼 채우는거

# raw_datasets의 모든 예제에 위 함수 적용하여 토큰화된 데이터 셋 생성 후 batched를 활용해 데이터셋을 배치로 처리
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# DataCollatorWithPadding 클래스 활용 데이터를 배치로 그룹화, 배치 내에 패딩 추가하여 데이터 처리 객체 생성
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [ ]:
pip install transformers[torch]

In [ ]:
pip install accelerate -U

In [ ]:
pip install accelerate

In [21]:
# transformers에서 TrainingArguments 클래스를 가져온다
# TrainingArguments : 모델 훈련 시 필요한 여러 매개변수와 설정을 구성하는데 사용
from transformers import TrainingArguments
# TrainingArguments 사용해서 모델 훈련 설정 구성
# "test-trainer"라는 이름의 설정파일로 저장 ( 모델 훈련에 필요한 매개변수와 하이퍼파라미터 포함)
# 매개변수 : 학습률, 에포크 수, 배치 크기, 저장디렉토리 등
# 그 후 training_args 변수에 저장
training_args = TrainingArguments("test-trainer")

In [22]:
# AutoModelForSequenceClassification : 시퀀스 분류 작업을 위한 사전 훈련된 모델을 초기화하는데 사용
from transformers import AutoModelForSequenceClassification

# AutoModelForSequenceClassification 사용해서 모델 초기화
# from_pretrained 함수 활용해서 checkpoint변수에서 지정한 사전 훈련된 모델을 불러온다
# num_labels를 활용해 예측해야하는 클래스 레이블의 수 지정 (이진분류할예정)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Trainer : 모델 훈련을 관리하고 실행하는데 사용
from transformers import Trainer

trainer = Trainer(
    model, # 위에서 정한 모델
    training_args, # 위에서 정함
    train_dataset=tokenized_datasets["train"],# train_dataset에는 훈련 데이터셋 전달 tokenized_datasets에서 "train"키를 사용하여 가져온다
    eval_dataset=tokenized_datasets["validation"], # eval_dataset에는 검증 데이터셋 전달 validation키를 사용하여 가져온다.
    data_collator=data_collator, # data_collator 변수에는 데이터 처리 객체 전달/ 이 객체는 배치 데이터를 생성하고 패딩을 추가하는 역할
    tokenizer=tokenizer, # 모델 입력 데이터를 토큰화하는데 사용되는 토크나이저 tokenizer에전달
)

In [24]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.505100
1000,0.244600


TrainOutput(global_step=1377, training_loss=0.29600822986167846, metrics={'train_runtime': 194.8512, 'train_samples_per_second': 56.474, 'train_steps_per_second': 7.067, 'total_flos': 405324636337200.0, 'train_loss': 0.29600822986167846, 'epoch': 3.0})

In [25]:
# tokenized_datasets["validation"]에 대한 예측 수행
# 모델이 검증 데이터셋을 사용하여 예측 생성하고 predictions 변수에 예측 결과 저장
predictions = trainer.predict(tokenized_datasets["validation"])
# 예측 결과와 레이블의 형태 출력
# 처음은 예측 두번째는 실제 레이블 형태
# 일반적으로 predictions.predictions.shape는 모델이 각 클래스에 대한 점수를 나타내며 이를 사용해 예측 클래스를 결정하거나 분류성능을 평가함
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [26]:
import numpy as np

# np.argmax(predictions.predictions)를 통해 각 예제에 대한 최우도 클래스 계산
preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
pip install evaluate

In [29]:
import evaluate

metric = evaluate.load("glue", "mrpc")
# 평가 메트릭 계산
# 이 매서드에 예측,실제 레이블 전달됨
metric.compute(predictions=preds, references=predictions.label_ids)
# 성능 측정하는데에 사용
# f1 : 정밀도와 재현율의 조화 평균 (모델이 얼마나 정확하게 긍정과 부정클래스를 분류하는지)

{'accuracy': 0.8676470588235294, 'f1': 0.9065743944636677}

In [33]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 위의과정들을 compute_metircs() 함수에 담는 과정?

In [31]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
# compute_metrics는 함수인데 eval_preds 매개변수를 안받고도 실행이 되나?

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
trainer.train() # No log가 왜 나올까?

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.470278,0.779412,0.830827
2,0.551100,0.661137,0.745098,0.842424
3,0.395800,0.584061,0.848039,0.891608


TrainOutput(global_step=1377, training_loss=0.42026507032030114, metrics={'train_runtime': 214.421, 'train_samples_per_second': 51.32, 'train_steps_per_second': 6.422, 'total_flos': 405540469624800.0, 'train_loss': 0.42026507032030114, 'epoch': 3.0})